A general overview and code of how the merging of the calculated points goes.

For the two groupings we're merging, the first, varaibles with no number at the end, we have 3 parings of ids where we have the information of their combined cost and combined points. For the second group, variables that end in `2`, there are four groups of ids, costs, and points. We're working in deminsions here, so it's important to keep in mind that as long as the indicies of the values associated with the people are legal, then we're good.

As a note, look and go through the `example_brisk_optimize` notebook first. It's a simlar calculations, but easier to understand than this.

In [1]:
import numpy as np
import itertools
import pandas as pd
import ast #used for conversion of the id list pandas is reading
df = pd.read_csv('data/example_data.csv', converters={0:lambda x: ast.literal_eval(x)})
df

,ids,sal,pts,pos
0,"[0, 4]",5,3,PG
1,"[1, 5]",7,5,PG
2,"[2, 6]",8,6,PG
3,"[3, 7]",9,7,PG
4,"[8, 13]",2,2,SG
5,"[9, 14]",3,4,SG
6,"[10, 15]",5,6,SG
7,"[11, 16]",6,6,SG
8,"[12, 17]",8,7,SG


In [2]:
pgdf = df[df.pos=='PG']
pgdf

,ids,sal,pts,pos
0,"[0, 4]",5,3,PG
1,"[1, 5]",7,5,PG
2,"[2, 6]",8,6,PG
3,"[3, 7]",9,7,PG


In [3]:
sgdf = df[df.pos=='SG']
sgdf

,ids,sal,pts,pos
4,"[8, 13]",2,2,SG
5,"[9, 14]",3,4,SG
6,"[10, 15]",5,6,SG
7,"[11, 16]",6,6,SG
8,"[12, 17]",8,7,SG


In [4]:
pgids = np.array(pgdf.ids.tolist())
pgsals = np.array(pgdf.sal.tolist())
pgpts = np.array(pgdf.pts.tolist())
print("Ids:", pgids)
print("Salaries:", pgsals)
print("Points:", pgpts)

Ids: [[0 4]
 [1 5]
 [2 6]
 [3 7]]
Salaries: [5 7 8 9]
Points: [3 5 6 7]


In [5]:
sgids = np.array(sgdf.ids.tolist())
sgsals = np.array(sgdf.sal.tolist())
sgpts = np.array(sgdf.pts.tolist())
print("SG Ids:", sgids)
print("SG Salaries:", sgsals)
print("SG Points:", sgpts)

SG Ids: [[ 8 13]
 [ 9 14]
 [10 15]
 [11 16]
 [12 17]]
SG Salaries: [2 3 5 6 8]
SG Points: [2 4 6 6 7]


In [6]:
ids_comb = np.array([np.concatenate((x,y)) for x,y in list(itertools.product(pgids, sgids))])
ids_comb

array([[ 0,  4,  8, 13],
       [ 0,  4,  9, 14],
       [ 0,  4, 10, 15],
       [ 0,  4, 11, 16],
       [ 0,  4, 12, 17],
       [ 1,  5,  8, 13],
       [ 1,  5,  9, 14],
       [ 1,  5, 10, 15],
       [ 1,  5, 11, 16],
       [ 1,  5, 12, 17],
       [ 2,  6,  8, 13],
       [ 2,  6,  9, 14],
       [ 2,  6, 10, 15],
       [ 2,  6, 11, 16],
       [ 2,  6, 12, 17],
       [ 3,  7,  8, 13],
       [ 3,  7,  9, 14],
       [ 3,  7, 10, 15],
       [ 3,  7, 11, 16],
       [ 3,  7, 12, 17]])

We can see that the array below has all the different ways we can combined those salary numbers

In [7]:
sal_comb = np.array(list(itertools.product(pgsals, sgsals)))
sal_comb

array([[5, 2],
       [5, 3],
       [5, 5],
       [5, 6],
       [5, 8],
       [7, 2],
       [7, 3],
       [7, 5],
       [7, 6],
       [7, 8],
       [8, 2],
       [8, 3],
       [8, 5],
       [8, 6],
       [8, 8],
       [9, 2],
       [9, 3],
       [9, 5],
       [9, 6],
       [9, 8]])

And now we can see that those values have been added together. Now we know the combined salaries of those groups of players

In [8]:
sal_sum = sal_comb.sum(axis=1)
sal_sum

array([ 7,  8, 10, 11, 13,  9, 10, 12, 13, 15, 10, 11, 13, 14, 16, 11, 12,
       14, 15, 17])

Like above, were calculating the product of the `points` and `points2` variables. Key point is that `itertools.product` returns the values in the same order. If that wasn't the case, this fails.

In [9]:
points_comb = np.array(list(itertools.product(pgpts, sgpts)))
points_comb

array([[3, 2],
       [3, 4],
       [3, 6],
       [3, 6],
       [3, 7],
       [5, 2],
       [5, 4],
       [5, 6],
       [5, 6],
       [5, 7],
       [6, 2],
       [6, 4],
       [6, 6],
       [6, 6],
       [6, 7],
       [7, 2],
       [7, 4],
       [7, 6],
       [7, 6],
       [7, 7]])

In [10]:
points_sum = points_comb.sum(axis=1)
points_sum

array([ 5,  7,  9,  9, 10,  7,  9, 11, 11, 12,  8, 10, 12, 12, 13,  9, 11,
       13, 13, 14])

For the third time, we're grouping the ids together in the same order. The concatenate makes the array have only two dimensions which is what we want for now.

In [11]:
test_salaries = np.array([9,10,11,12])
test_salaries

array([ 9, 10, 11, 12])

When we run the `broadcast_to`, we can see that with the following shape, each row is a test for each salary. Keep that in mind.

In [12]:
test_salaries_full = np.broadcast_to(test_salaries, (sal_sum.size, test_salaries.size)).T
test_salaries_full

array([[ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11],
       [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12]])

In [13]:
ids_comb_full = np.broadcast_to(ids_comb, (test_salaries.size, points_sum.size, 4))
ids_comb_full

array([[[ 0,  4,  8, 13],
        [ 0,  4,  9, 14],
        [ 0,  4, 10, 15],
        [ 0,  4, 11, 16],
        [ 0,  4, 12, 17],
        [ 1,  5,  8, 13],
        [ 1,  5,  9, 14],
        [ 1,  5, 10, 15],
        [ 1,  5, 11, 16],
        [ 1,  5, 12, 17],
        [ 2,  6,  8, 13],
        [ 2,  6,  9, 14],
        [ 2,  6, 10, 15],
        [ 2,  6, 11, 16],
        [ 2,  6, 12, 17],
        [ 3,  7,  8, 13],
        [ 3,  7,  9, 14],
        [ 3,  7, 10, 15],
        [ 3,  7, 11, 16],
        [ 3,  7, 12, 17]],

       [[ 0,  4,  8, 13],
        [ 0,  4,  9, 14],
        [ 0,  4, 10, 15],
        [ 0,  4, 11, 16],
        [ 0,  4, 12, 17],
        [ 1,  5,  8, 13],
        [ 1,  5,  9, 14],
        [ 1,  5, 10, 15],
        [ 1,  5, 11, 16],
        [ 1,  5, 12, 17],
        [ 2,  6,  8, 13],
        [ 2,  6,  9, 14],
        [ 2,  6, 10, 15],
        [ 2,  6, 11, 16],
        [ 2,  6, 12, 17],
        [ 3,  7,  8, 13],
        [ 3,  7,  9, 14],
        [ 3,  7, 10, 15],
        [ 

In [14]:
sal_sum_full = np.broadcast_to(sal_sum,(test_salaries.size, sal_sum.size))
sal_sum_full

array([[ 7,  8, 10, 11, 13,  9, 10, 12, 13, 15, 10, 11, 13, 14, 16, 11,
        12, 14, 15, 17],
       [ 7,  8, 10, 11, 13,  9, 10, 12, 13, 15, 10, 11, 13, 14, 16, 11,
        12, 14, 15, 17],
       [ 7,  8, 10, 11, 13,  9, 10, 12, 13, 15, 10, 11, 13, 14, 16, 11,
        12, 14, 15, 17],
       [ 7,  8, 10, 11, 13,  9, 10, 12, 13, 15, 10, 11, 13, 14, 16, 11,
        12, 14, 15, 17]])

In [15]:
points_sum_full = np.broadcast_to(points_sum,(test_salaries.size, points_sum.size))
points_sum_full

array([[ 5,  7,  9,  9, 10,  7,  9, 11, 11, 12,  8, 10, 12, 12, 13,  9,
        11, 13, 13, 14],
       [ 5,  7,  9,  9, 10,  7,  9, 11, 11, 12,  8, 10, 12, 12, 13,  9,
        11, 13, 13, 14],
       [ 5,  7,  9,  9, 10,  7,  9, 11, 11, 12,  8, 10, 12, 12, 13,  9,
        11, 13, 13, 14],
       [ 5,  7,  9,  9, 10,  7,  9, 11, 11, 12,  8, 10, 12, 12, 13,  9,
        11, 13, 13, 14]])

At this point, note the shapes of `sal_sum_full`, `points_sum_full`, `test_salaries_full`, and `ids_comb_full`:

In [16]:
print("sal_sum_full:     ", sal_sum_full.shape)
print("points_sum_full:   ", points_sum_full.shape)
print("test_salaries_full:", test_salaries_full.shape)
print("ids_comb_full:     ", ids_comb_full.shape)

sal_sum_full:      (4, 20)
points_sum_full:    (4, 20)
test_salaries_full: (4, 20)
ids_comb_full:      (4, 20, 4)


The key work before was to get these deminsions to match up so we can do direct comparison. As an example, we can pick (x,y) coords as (2,4) and see that the information matches up. Remember, the reason that `ids_comb_full` has that 3rd dimension is because we're combining groups of 4 player ids.

In [17]:
x,y = 2,4
print("Indexes: ", ids_comb_full[x][y])
print("Salaries: ", sal_sum_full[x][y])
print("Points: ", points_sum_full[x][y])

Indexes:  [ 0  4 12 17]
Salaries:  13
Points:  10


In [18]:
valids = sal_sum_full <= test_salaries_full
valids

array([[ True,  True, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False],
       [ True,  True,  True, False, False,  True,  True, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False],
       [ True,  True,  True,  True, False,  True,  True, False, False,
        False,  True,  True, False, False, False,  True, False, False,
        False, False],
       [ True,  True,  True,  True, False,  True,  True,  True, False,
        False,  True,  True, False, False, False,  True,  True, False,
        False, False]])

In [19]:
possibilities = points_sum_full * valids
possibilities

array([[ 5,  7,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 5,  7,  9,  0,  0,  7,  9,  0,  0,  0,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 5,  7,  9,  9,  0,  7,  9,  0,  0,  0,  8, 10,  0,  0,  0,  9,
         0,  0,  0,  0],
       [ 5,  7,  9,  9,  0,  7,  9, 11,  0,  0,  8, 10,  0,  0,  0,  9,
        11,  0,  0,  0]])

In [20]:
top_inds = possibilities.argmax(axis=1)
top_inds

array([ 1,  2, 11,  7])

In [21]:
row_selectors = np.arange(test_salaries.size)
row_selectors

array([0, 1, 2, 3])

You might be looking at this oddly, as to what the `top_inds` and `row_selectors` are for. This brings up an important point with numpy where the actual dimensions of the matricies that are being used isn't all important, as long as you know how to extract the information you're looking for. I could have reshaped these however I wanted as long as I used the same shape for all the matricies.

In [22]:
max_points = points_sum_full[row_selectors, top_inds]
max_points

array([ 7,  9, 10, 11])

In [23]:
max_sals = sal_sum_full[row_selectors, top_inds]
max_sals

array([ 8, 10, 11, 12])

In [24]:
max_inds = ids_comb_full[row_selectors, top_inds]
max_inds

array([[ 0,  4,  9, 14],
       [ 0,  4, 10, 15],
       [ 2,  6,  9, 14],
       [ 1,  5, 10, 15]])

In [25]:
test_salaries

array([ 9, 10, 11, 12])

I included the test_salaries above as a reminder of which index in the array the test salary of 11 was.
When you run the cell below, you'll see that those values match with what was calculated in the brisk solution notebook.

In [26]:
print("Top players:", max_inds[2]) #2, because that's where test_salaries is 11
print("Top points:", max_points[2])
print("Combined salaries:", max_sals[2])

Top players: [ 2  6  9 14]
Top points: 10
Combined salaries: 11
